# Efetuar a consulta ao Neo4j, gerar objeto graph_data

In [3]:
from collections import defaultdict
from neo4j import GraphDatabase

class Neo4jQuery:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def get_graph_data(self):
        with self._driver.session() as session:
            # Consulta para incluir filtros e lógicas específicas para gerar o grafo
            result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
            nodes = {}
            links = []
            node_labels_count = defaultdict(int)  # Para contar a frequência dos rótulos dos nós
            relationship_types_count = defaultdict(int)  # Para contar a frequência dos tipos de relacionamentos

            for record in result:
                start_node = record['n']
                end_node = record['m']
                relationship = record['r']
                
                # Adiciona nós ao dicionário se ainda não estiverem presentes
                if start_node.element_id not in nodes:
                    nodes[start_node.element_id] = {'id': start_node.element_id, 'label': list(start_node.labels)[0]}
                    node_labels_count[list(start_node.labels)[0]] += 1
                if end_node.element_id not in nodes:
                    nodes[end_node.element_id] = {'id': end_node.element_id, 'label': list(end_node.labels)[0]}
                    node_labels_count[list(end_node.labels)[0]] += 1
                
                # Adiciona o relacionamento à lista de links
                links.append({
                    'source': start_node.element_id,
                    'target': end_node.element_id,
                    'type': relationship.type,
                    'properties': dict(relationship)
                })
                relationship_types_count[relationship.type] += 1

            graph_data = {
                'nodes': list(nodes.values()),
                'links': links,
                'node_count': len(nodes),
                'link_count': len(links),
                'node_labels_count': dict(node_labels_count),  # Converte o defaultdict para um dicionário normal
                'relationship_types_count': dict(relationship_types_count)  # Converte o defaultdict para um dicionário normal                
            }

            # Converte o dicionário de nós para uma lista para uso em d3.js
            return graph_data

# Uso da classe:
neo4j_query = Neo4jQuery('neo4j://localhost:7687', 'neo4j', 'password')
graph_data = neo4j_query.get_graph_data()
neo4j_query.close()

# Após a chamada, graph_data agora contém a contagem de nós e arestas
print(f"   Quantidade de nós: {graph_data['node_count']}")
# Defina a ordem específica das chaves
ordered_node_labels = ['ÁrvoreCNPq', 'GrandeÁrea', 'Área', 'Subárea', 'Especialidade']

# Imprima o conteúdo de graph_data['node_labels_count'] na ordem especificada
for label in ordered_node_labels:
    count = graph_data['node_labels_count'].get(label, 0)  # Obtém a contagem, ou 0 se a chave não existir
    print(f"{label:>20}: {count}")

print(f"\n Contagem de arestas: {graph_data['link_count']}")
# Defina a ordem específica das chaves
ordered_edge_labels = ['CONTÉM', 'CONTÉM_ÁREA', 'CONTÉM_SUBÁREA', 'CONTÉM_ESPECIALIDADE']

# Imprima o conteúdo de graph_data['node_labels_count'] na ordem especificada
for label in ordered_edge_labels:
    count = graph_data['relationship_types_count'].get(label, 0)  # Obtém a contagem, ou 0 se a chave não existir
    print(f"{label:>20}: {count}")

import os
import json
import webbrowser
from IPython.display import HTML, display

# Configuração do NeoVis
config = {
    "container_id": "viz",
    "server_url": "bolt://localhost:7687",
    "server_user": "neo4j",
    "server_password": "password",
    "initial_cypher": "MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 10000",
    "physics": {
        "stabilization": {
            "enabled": True,
            "iterations": 5
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -5,
            "centralGravity": 0.005,
            "springLength": 250,
            "springConstant": 0.15
        },
        "maxVelocity": 250,
        "minVelocity": 5,
        "timestep": 60
    }
}

# Código HTML/JS ampliado para incorporar o grafo com barra de progresso e controles interativos
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Visualização do Grafo</title>
    <style>
        /* Aqui você pode adicionar estilos para a barra de progresso e sliders */
        #controls {{
            margin-bottom: 10px;
        }}
        .slider {{
            width: 200px;
        }}
    </style>
</head>
<body>
    <div id="controls">
        <label>Gravitational Constant: <input type="range" id="gravitationalConstantSlider" class="slider" min="-50" max="0" value="-5"></label>
        <label>Central Gravity: <input type="range" id="centralGravitySlider" class="slider" min="0" max="0.01" step="0.001" value="0.005"></label>
        <label>Spring Length: <input type="range" id="springLengthSlider" class="slider" min="50" max="500" value="250"></label>
        <label>Spring Constant: <input type="range" id="springConstantSlider" class="slider" min="0" max="1" step="0.05" value="0.15"></label>
        <!-- Outros sliders conforme necessário -->
    </div>
    <div id="{config['container_id']}" style="height: 800px; width:100%;"></div>
    <progress id="progressBar" value="0" max="100" style="width:100%;"></progress>
    <script type="text/javascript" src="https://cdn.neo4jlabs.com/neovis.js/v1.5.0/neovis.js"></script>
    <script type="text/javascript">
        var config = {json.dumps(config)};
        var viz;

        function updateGraphPhysics(option, value) {{
            config.physics.forceAtlas2Based[option] = value;
            viz.setOptions({{ physics: config.physics }});
            viz.reload();
        }}

        function draw() {{
            viz = new NeoVis.default(config);
            viz.registerOnEvent('completed', function() {{
                document.getElementById('progressBar').value = 100; // Ajusta a barra de progresso para completo
            }});
            viz.render();

            document.getElementById('gravitationalConstantSlider').addEventListener('input', function(e) {{
                updateGraphPhysics('gravitationalConstant', parseFloat(e.target.value));
            }});

            document.getElementById('centralGravitySlider').addEventListener('input', function(e) {{
                updateGraphPhysics('centralGravity', parseFloat(e.target.value));
            }});

            document.getElementById('springLengthSlider').addEventListener('input', function(e) {{
                updateGraphPhysics('springLength', parseFloat(e.target.value));
            }});

            document.getElementById('springConstantSlider').addEventListener('input', function(e) {{
                updateGraphPhysics('springConstant', parseFloat(e.target.value));
            }});
            // Repita para outros sliders conforme necessário
        }}

        window.onload = draw;
    </script>
</body>
</html>
"""

# Exibe o HTML no notebook (não funciona)
# display(HTML(html_template))

# Salve o conteúdo em um arquivo HTML
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

   Quantidade de nós: 1286
          ÁrvoreCNPq: 1
          GrandeÁrea: 8
                Área: 76
             Subárea: 340
       Especialidade: 861

 Contagem de arestas: 1285
              CONTÉM: 8
         CONTÉM_ÁREA: 76
      CONTÉM_SUBÁREA: 340
CONTÉM_ESPECIALIDADE: 861


True

# PyGraphistry

In [6]:
### MINIMALIST
#!pip install --user pandas
#!pip install --user graphistry[bolt]

### ADVANCED:
### If you already have the neo4j python driver, you can leave out '[bolt]':
### !pip3 install --user graphistry

### If you already have graphistry but not neo4j, you can reuse your existing graphistry:
### !pip install --user neo4j

import pandas as pd
import neo4j # just for testing
from neo4j import GraphDatabase # for data loader
import graphistry
print('neo4j', neo4j.__version__)
print('graphistry', graphistry.__version__)

neo4j 5.2.1
graphistry 0.29.7


# NeoVis

In [ ]:
# Funcionando, porém sem barra de progresso e controladores interativos
import os
import json
import webbrowser
from IPython.display import HTML, display

# Configuração do NeoVis
config = {
    "container_id": "viz",
    "server_url": "bolt://localhost:7687",
    "server_user": "neo4j",
    "server_password": "password",
    "initial_cypher": "MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 10000",

    "physics": {
        "stabilization": {
            "enabled": True,
            "iterations": 5
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -5,
            "centralGravity": 0.005,
            "springLength": 250,
            "springConstant": 0.15
        },
        "maxVelocity": 250,
        "minVelocity": 5,
        "timestep": 60
    }
}

# Código HTML/JS para incorporar o NeoVis.js
html_template = f"""
<div id="{config['container_id']}" style="height: 900px; width:100%;"></div>
<script type="text/javascript" src="https://cdn.neo4jlabs.com/neovis.js/v1.5.0/neovis.js"></script>
<script type="text/javascript">
    var config = {json.dumps(config)};
    var viz;

    function draw() {{
        viz = new NeoVis.default(config);
        viz.render();
    }}
    
    // Aguarda a página carregar completamente antes de desenhar a visualização
    window.onload = draw;
</script>
"""

# Exibe o HTML no notebook (não funciona)
# display(HTML(html_template))

# Salve o conteúdo em um arquivo HTML
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

True

# Neovis

A identificação e visualização de comunidades em um grafo gerado pelo NeoVis.js dependem, em parte, da configuração de física (physics) da biblioteca de visualização utilizada, como o Vis.js, que é integrada ao NeoVis.js. 

A física controla como os nós interagem uns com os outros e com as arestas, o que afeta diretamente a distribuição dos nós e a aparência das comunidades no grafo.

Para permitir que as comunidades tenham mais iterações e busquem um melhor ajuste, pode-se configurar as opções de física no NeoVis.js para aumentar o número de iterações do algoritmo de layout. 

No entanto, o NeoVis.js tem limitações de configuração pré-definidas e pode não expor todas as opções avançadas de física diretamente. 

Em muitos casos, um ajuste fino requer a manipulação direta do Vis.js ou o uso de algoritmos de detecção de comunidades no lado do banco de dados Neo4j antes da visualização.

## Configurar física da simulação

In [2]:
# !pip3 install ipywidgets jupyterlab
# import os
# os.listdir('./../../output/')

# Configuração do NeoVis
config = {
    "container_id": "viz",
    "server_url": "bolt://localhost:7687",
    "server_user": "neo4j",
    "server_password": "password",
    "initial_cypher": "MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 10000",

    "physics": {
        "stabilization": {
            "enabled": True,
            "iterations": 5
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -5,
            "centralGravity": 0.005,
            "springLength": 250,
            "springConstant": 0.15
        },
        "maxVelocity": 250,
        "minVelocity": 5,
        "timestep": 60
    }
}

In [3]:
# Renderiza a barra de progresso, mas não renderiza o grafo
import os
import json
import webbrowser
from IPython.display import HTML, display

# Código HTML/JS para incorporar o NeoVis.js
html_template = f"""
<div id="{config['container_id']}" style="height: 900px; width:100%;"></div>
<!-- Barra de progresso -->
<div id="progress-container" style="width: 100%; background-color: #eee;">
    <div id="progressBar" style="width: 0%; height: 30px; background-color: #76A7FA;">
        0%
    </div>
</div>
<script type="text/javascript" src="https://cdn.neo4jlabs.com/neovis.js/v1.5.0/neovis.js"></script>
<script type="text/javascript">
    var config = {json.dumps(config)};
    var viz;

    function draw() {{
        viz = new NeoVis.default(config);
        viz.render();
    
    // Aguarda a página carregar completamente antes de desenhar a visualização
    window.onload = draw;

    // Função para atualizar a barra de progresso
    function updateProgressBar(percent) {{
        var progressBar = document.getElementById('progressBar');
        progressBar.style.width = percent + '%';
        progressBar.textContent = percent.toFixed(2) + '%';
    }}   
    
    // Escutando eventos de estabilização e atualizando a barra de progresso
        viz.registerOnEvent('stabilizationProgress', function(params) {{
            var widthPercent = (params.iterations / params.total) * 100;
            updateProgressBar(widthPercent);
        }});

        viz.registerOnEvent('stabilizationIterationsDone', function() {{
            updateProgressBar(100);
        }});     
            
    }}    
    
</script>
"""

# Exibe o HTML no notebook (não funciona)
# display(HTML(html_template))

# Salve o conteúdo em um arquivo HTML
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

True

## Renderizar na célula do Jupyter

In [4]:
import os
import json
import webbrowser
from IPython.display import HTML, Javascript, display

# Convertendo configuração para string JSON
config_str = json.dumps(config)

# HTML e JS template com chaves duplas para literal no JavaScript
html_template = f"""
<div id="Árvore do Conhecimento CNPq" style="height: 800px; width: 100%;"></div>
<div id="progressContainer" style="width: 100%; background-color: #ddd;">
    <div id="progressBar" style="width: 1%; height: 30px; background-color: #4CAF50; text-align: center; line-height: 30px; color: white;">0%</div>
</div>
<script type="text/javascript" src="https://cdn.neo4jlabs.com/neovis.js/v1.5.0/neovis.js"></script>
<script type="text/javascript">
  document.addEventListener('DOMContentLoaded', function() {{
    // Garante que o código é executado após o carregamento completo da página

    // Verifica se o NeoVis está definido
    if (typeof NeoVis === 'undefined') {{
      alert('NeoVis não está carregado!');
      return;
    }}

    // Cria uma instância do NeoVis
    var config = {config_str};
    var viz = new NeoVis(config);
    viz.render();

    // Função para atualizar a barra de progresso
    function updateProgressBar(percent) {{
        var progressBar = document.getElementById('progressBar');
        progressBar.style.width = percent + '%';
        progressBar.textContent = percent.toFixed(2) + '%';
    }}

    // Escutando eventos de estabilização e atualizando a barra de progresso
    viz.registerOnEvent('stabilizationProgress', function(params) {{
        var widthPercent = (params.iterations / params.total) * 100;
        updateProgressBar(widthPercent);
    }});

    viz.registerOnEvent('stabilizationIterationsDone', function() {{
        updateProgressBar(100);
    }});
  }});
</script>
"""

# Exibe o template no Jupyter Notebook (não funcionando)
# display(HTML(html_template))

## Salve o conteúdo em um arquivo HTML (Ok)
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

True

In [ ]:
import os
os.listdir('./../../../../../../mak/node_modules/neovis.js')
# <script src="./../../../../../../mak/node_modules/neovis.js"></script>

In [ ]:
from IPython.display import IFrame, display_html

# Crie um arquivo HTML temporário com seu conteúdo de visualização
html_content = f"""
<div id="{config['container_id']}" style="height: 900px; width:100%;"></div>
<!-- outros elementos, como a barra de progresso, podem ser incluídos aqui -->
<script type="text/javascript" src="/home/mak/node_modules/neovis.js/dist/neovis.js"></script>
<script type="text/javascript">
// Seu código JavaScript aqui
</script>
"""
path_to_html_file = 'temp_visualization.html'
with open(path_to_html_file, 'w') as f:
    f.write(html_content)

# Renderizar o arquivo HTML em um iframe dentro do notebook Jupyter
display_html(IFrame(src=path_to_html_file, width='100%', height='800px'))

In [ ]:
import os
import json
import webbrowser
from IPython.display import HTML, Javascript, display

# Convertendo configuração para string JSON
config_str = json.dumps(config)

# HTML e JS template com chaves duplas para literal no JavaScript
html_template = f"""
<div id="Árvore Conhecimento CNPq" style="height: 800px; width: 100%;"></div>
<div id="progressContainer" style="width: 100%; background-color: #ddd;">
    <div id="progressBar" style="width: 1%; height: 30px; background-color: #4CAF50; text-align: center; line-height: 30px; color: white;">0%</div>
</div>
<script type="text/javascript" src="/home/mak/node_modules/neovis.js/dist/neovis.js"></script>
<script type="text/javascript">
  document.addEventListener('DOMContentLoaded', function() {{
    // Garante que o código é executado após o carregamento completo da página

    // Verifica se o NeoVis está definido
    if (typeof NeoVis === 'undefined') {{
      alert('NeoVis não está carregado!');
      return;
    }}

    // Cria uma instância do NeoVis
    var config = {config_str};
    var viz = new NeoVis(config);
    viz.render();

    // Função para atualizar a barra de progresso
    function updateProgressBar(percent) {{
        var progressBar = document.getElementById('progressBar');
        progressBar.style.width = percent + '%';
        progressBar.textContent = percent.toFixed(2) + '%';
    }}

    // Escutando eventos de estabilização e atualizando a barra de progresso
    viz.registerOnEvent('stabilizationProgress', function(params) {{
        var widthPercent = (params.iterations / params.total) * 100;
        updateProgressBar(widthPercent);
    }});

    viz.registerOnEvent('stabilizationIterationsDone', function() {{
        updateProgressBar(100);
    }});
  }});
</script>
"""

# Exibe o template no Jupyter Notebook (não funcionando)
# display(HTML(html_template))

# Salve o conteúdo em um arquivo HTML
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

## Com controles interativos no HTML

In [ ]:
import os
import json
import webbrowser
from IPython.display import HTML, display

# Configurações do NeoVis - substitua com suas credenciais/configurações do Neo4j
config = {
    "container_id": "viz",
    "server_url": "bolt://localhost:7687",
    "server_user": "neo4j",
    "server_password": "password",
    "initial_cypher": "MATCH (n)-[r]->(m) RETURN n,r,m LIMIT 10000",

    "physics": {
        "stabilization": {
            "enabled": True,
            "iterations": 50000000
        },
        "solver": "forceAtlas2Based",
        "forceAtlas2Based": {
            "gravitationalConstant": -5,
            "centralGravity": 0.005,
            "springLength": 150,
            "springConstant": 0.15
        },
        "maxVelocity": 250,
        "minVelocity": 5,
        "timestep": 60
    }
}

# Código HTML/JS para incorporar o NeoVis.js
html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Interactive Graph Controls</title>
</head>
<body>
    <!-- Container for the visualization -->
    <div id="mynetwork"></div>

    <!-- Controls for physics parameters -->
    <div id="physicsControls">
        <label for="stabilizationIterations">Stabilization Iterations:</label>
        <input type="range" id="stabilizationIterations" min="10" max="20000000" step="1000">
        
        <label for="timestep">Timestep:</label>
        <input type="range" id="timestep" min="0.1" max="1" step="0.1">
        
        <button onclick="updatePhysics()">Update Physics</button>
    </div>

    <!-- Include required libraries -->
    <script type="text/javascript" src="path/to/vis-network.min.js"></script>
    <script type="text/javascript" src="path/to/neovis.js"></script>

    <script>
        // Initialization of the graph with NeoVis
        var config = {{
            container_id: "mynetwork",
            physics: {{
                // Default physics options
                stabilization: {{ iterations: 1000 }},
                timestep: 0.5
            }}
        }};
        var viz = new NeoVis(config);
        viz.render();

        // Function to update the physics of the network
        function updatePhysics() {{
            var stabilizationIterations = document.getElementById('stabilizationIterations').value;
            var timestepValue = document.getElementById('timestep').value;

            // Updating the physics options
            viz.setOptions({{
                physics: {{
                    stabilization: {{ iterations: parseInt(stabilizationIterations) }},
                    timestep: parseFloat(timestepValue)
                }}
            }});

            // Restarting the simulation with the new physics options
            viz.stabilize();
        }}
    </script>
</body>
</html>
"""

# Exibe o HTML no notebook
# display(HTML(html_template))

# # Salve o conteúdo em um arquivo HTML
folder='./../../output/'
file_path = 'neovis_graph.html'
with open(folder+file_path, 'w') as file:
    file.write(html_template)

# Abrindo o arquivo HTML em uma nova aba do navegador padrão
webbrowser.open('file://' + os.path.realpath(folder+file_path))

# Cytoscape

In [ ]:
# !pip3 install --upgrade pip
# !pip3 install cyjupyter
# !pip3 install --upgrade notebook --user

In [ ]:
from cyjupyter import Cytoscape
import json

# Assuming `graph_data` is your graph data structured as Cytoscape.js expects
cyto_graph = Cytoscape(data=graph_data, visual_style=style, layout_name='cola')

# Display in Jupyter Notebook
display(cyto_graph)

# Renderizar em HTML

In [ ]:
html_template = f"""
<style>
#graph svg {{
  display: block;
  margin: auto;
  border: 1px solid #ccc; /* Optional: adds a border around the SVG */
}}
</style>
<div id="graph"></div>
<script src="https://d3js.org/d3.v5.min.js"></script>
<script>

var graph = {graph_data};

var width = 1600,
    height = 900;

var svg = d3.select("#graph").append("svg")
    .attr("width", width)
    .attr("height", height);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().id(function(d) {{ return d.id; }}).distance(10)) // Smaller distance means tighter links
    .force("charge", d3.forceManyBody().strength(-0.2)) // Less negative strength means less repulsion
    .force("center", d3.forceCenter(width/2, height/2))
    .force("collide", d3.forceCollide().radius(function(d) {{ return d.r + 8; }}).iterations(8))
    .force("box", boundingBoxForce());

var link = svg.append("g")
    .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
    .attr("stroke-width", function(d) {{ return Math.sqrt(d.value) || 2; }})
    .attr("stroke", "#999")
    .attr("stroke-opacity", 0.6);

var node = svg.append("g")
    .attr("class", "nodes")
    .selectAll("circle")
    .data(graph.nodes)
    .enter().append("circle")
    .attr("r", 5)
    .attr("fill", "blue");

node.append("title")
    .text(function(d) {{ return d.label; }});

simulation
    .nodes(graph.nodes)
    .on("tick", ticked);

simulation.force("link")
    .links(graph.links);

function ticked() {{
    link
        .attr("x1", function(d) {{ return d.source.x; }})
        .attr("y1", function(d) {{ return d.source.y; }})
        .attr("x2", function(d) {{ return d.target.x; }})
        .attr("y2", function(d) {{ return d.target.y; }});

    node
        .attr("cx", function(d) {{ return d.x; }})
        .attr("cy", function(d) {{ return d.y; }});
}}

// Function to keep nodes inside the SVG area
function boundingBoxForce() {{
    var box = {{x1: 0, y1: 0, x2: width, y2: height}};
    function force() {{
        for (let i = 0, n = graph.nodes.length, node; i < n; ++i) {{
            node = graph.nodes[i];
            node.x = Math.max(box.x1, Math.min(node.x, box.x2));
            node.y = Math.max(box.y1, Math.min(node.y, box.y2));
        }}
    }}
    return force;
}}

</script>
"""

# Python code to write the HTML template to a file and open it in a web browser
import webbrowser
import os

with open('graph_visualization.html', 'w') as f:
    f.write(html_template)
webbrowser.open('file://' + os.path.realpath('graph_visualization.html'))



# PyVis

In [ ]:
# !pip3 install pyvis

In [ ]:
# ... previous code remains unchanged ...

# Assuming graph_data is a dictionary with 'nodes' and 'links' lists
graph_data = neo4j_query.get_graph_data()  # Replace with your actual method call

# ... previous code remains unchanged ...





# ... rest of the code remains unchanged ...


In [ ]:
from pyvis.network import Network
import os
import webbrowser

# Create a new PyVis network
net = Network(height="900px", width="1600px", bgcolor="#222222", font_color="white")

# Add nodes and edges to the network
for node in graph_data['nodes']:
    net.add_node(node['id'], label=node['label'], title=node['label'], color="blue")

# Here we corrected 'edges' to 'links'
for link in graph_data['links']:
    # Assuming 'weight' is the property you want to use for the edge value
    net.add_edge(link['source'], link['target'], value=link.get('properties', {}).get('weight', 1))  

# Set the physics layout of the network
net.toggle_physics(True)
net.show_buttons(filter_=['physics'])  # This line will add a physics configuration menu

# Generate and save the network
net.show('graph_visualization.html')

# Open the file in a web browser
webbrowser.open('file://' + os.path.realpath('graph_visualization.html'))

# Renderizar com WebGL para usar GPU

In [ ]:
// Insira aqui o seu `graph_data`, assegurando que está formatado corretamente para uso no three.js
var graph_data = {/*...*/};

// Inicialize a cena, a câmera e o renderizador do Three.js
var scene = new THREE.Scene();
var camera = new THREE.PerspectiveCamera(75, width / height, 0.1, 1000);
var renderer = new THREE.WebGLRenderer();
renderer.setSize(width, height);
document.getElementById('graph').appendChild(renderer.domElement);

// Crie materiais e geometrias para os nós e arestas
var nodeMaterial = new THREE.MeshBasicMaterial({ color: 0x0000ff });
var nodeGeometry = new THREE.SphereGeometry(1, 32, 32); // Esferas para representar nós

var nodesMesh = graph_data.nodes.map(node => {
  var mesh = new THREE.Mesh(nodeGeometry, nodeMaterial);
  // Posicione seus meshes de acordo com os dados do nó
  mesh.position.x = /*...*/;
  mesh.position.y = /*...*/;
  mesh.position.z = /*...*/;
  scene.add(mesh);
  return mesh;
});

var lineMaterial = new THREE.LineBasicMaterial({ color: 0x999999 });

graph_data.links.forEach(link => {
  var geometry = new THREE.Geometry();
  // Posicione a geometria de acordo com os dados do link
  geometry.vertices.push(
    new THREE.Vector3(/* x1 */, /* y1 */, /* z1 */),
    new THREE.Vector3(/* x2 */, /* y2 */, /* z2 */)
  );
  var line = new THREE.Line(geometry, lineMaterial);
  scene.add(line);
});

// Crie uma simulação semelhante à d3.forceSimulation
// Você precisará implementar sua própria versão de uma simulação de força ou encontrar uma biblioteca que possa fazer isso em 3D.

// Função de renderização que é chamada em cada quadro de animação
function animate() {
  requestAnimationFrame(animate);
  
  // Atualizações de simulação iriam aqui
  
  renderer.render(scene, camera);
}

// Inicie a animação
animate();


# Servidor personalizado

In [ ]:
from flask import Flask, jsonify
from neo4j import GraphDatabase

app = Flask(__name__)
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def get_graph_data():
    with driver.session() as session:
        result = session.run("MATCH (n)-[r]->(m) RETURN n, r, m")
        # Converta o resultado para JSON
        # ...

@app.route("/graph")
def graph():
    data = get_graph_data()
    return jsonify(data)

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<meta charset="utf-8">
<body>
<script src="https://d3js.org/d3.v6.min.js"></script>
<script>

// Carregar dados do servidor Flask
d3.json("http://localhost:5000/graph").then(function(graph) {
    // Use d3.js para criar a visualização com os dados recebidos
    // ...
});

</script>

In [ ]:
import webbrowser
import os

def render_d3_in_browser(html_str, filename='d3_visualization.html'):
    path = os.path.abspath(filename)
    with open(path, 'w') as f:
        f.write(html_str)
    webbrowser.open('file://' + path)

html_str = """
<div id="d3-example"></div>
<script src="https://d3js.org/d3.v5.min.js"></script>
<script>
// Seu código d3.js aqui
</script>
"""

render_d3_in_browser(html_str)
